In [1]:
import math
import torch
from torch.autograd.gradcheck import zero_gradients
from torch.autograd import grad

from projects.NeuralForceField.train import *
from projects.NeuralForceField.graphs import * 
from projects.NeuralForceField.tensorgrad import *

import numpy as np

In [2]:
# ethanol data 
train = np.load('/home/wwj/data/ethanol_ccsd_t-train.npz')
xyz_data = np.dstack((np.array([train.f.z]*1000).reshape(1000, -1, 1), np.array(train.f.R)))
force_data = train.f.F
energy_data = train.f.E.squeeze()

graph_data = load_graph_data(xyz_data=xyz_data, energy_data=energy_data, batch_size=2, cutoff=5.0,
                             force_data=force_data, au_flag=False, subtract_mean_flag = True)

In [3]:
# initialize parameters
par = dict()

par["n_filters"] = 256
par["n_gaussians"] = 32
par["n_atom_basis"] = 256
par["optim"] = 1e-4
par["scheduler"] = True
par["train_percentage"] = 0.8
par["T"] = 10
par["batch_size"] = 2
par["cutoff"] = 5.0 
par["max_epoch"] = 1000
par["trainable_gauss"] = True
par["rho"] = 0.1
par["eps"] = 1e-5

In [4]:
model = Model(par=par, graph_data=graph_data, device=3, job_name="hessian_test", graph_batching=False, root='./log/')

In [6]:
xyz, a, r, f, u, N = model.parse_batch(1)
xyz = xyz.reshape(-1, 9 ,3)
r = r.reshape(-1, 9)

# compute neural network hessian
hess = Neural_hess(xyz=xyz, r=r, model=model.model, device=3)

In [7]:
torch.eig(hess)

RuntimeError: invalid argument 3: A should be (non-empty) 2 dimensional at /opt/conda/conda-bld/pytorch_1535490206202/work/aten/src/THC/generic/THCTensorMathMagma.cu:246

In [8]:
hess.shape

torch.Size([2, 27, 27])

In [10]:
torch.eig(hess[0], eigenvectors=False, out=None)

(tensor([[-76.1882,   0.0000],
         [ 57.1081,   0.0000],
         [-54.1098,   0.0000],
         [ 41.8741,   0.0000],
         [-43.8381,   0.0000],
         [ 29.2810,   0.0000],
         [ 33.3801,   0.0000],
         [ 32.9945,   0.0000],
         [-28.9078,   0.0000],
         [-25.3484,   0.0000],
         [ 21.9584,   0.0000],
         [-20.4256,   0.0000],
         [ 14.0442,   0.0000],
         [ 10.9138,   0.0000],
         [-12.0176,   0.0000],
         [-10.5882,   0.0000],
         [  5.6840,   0.0000],
         [  5.4874,   0.0000],
         [ -2.4246,   0.0000],
         [  2.2946,   0.0000],
         [  1.8927,   0.0000],
         [ -0.8512,   0.0000],
         [  0.6206,   0.0000],
         [  0.1933,   0.0000],
         [ -0.0000,   0.0000],
         [ -0.0000,  -0.0000],
         [  0.0000,   0.0000]], device='cuda:3'), tensor([], device='cuda:3'))